In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import nibabel as nib
import nilearn
from nilearn import plotting
from nilearn.masking import apply_mask 
from nilearn.connectome import ConnectivityMeasure
from nilearn.input_data import NiftiMapsMasker
from glob import glob
from os.path import join
import matplotlib.pylab as plt

In [ ]:
data_dir = "C:\\Users\\sr1209\\Downloads\\nilearn_data"

# Here, I accidentally used two identical IDs (0021019), but oh well...
ids = ['0010042', '0010128', '0027034', '0021019', '0027037', '0021019']
group = [1,0,1,0,1,0]
n_subjects = len(ids)

In [ ]:
# Load file names
phenotypic = data_dir + '\adhd\ADHD200_40subs_motion_parameters_and_phenotypics.csv'
func_fns = [str(data_dir + '\\adhd\\data\\%s\\%s_rest_tshift_RPI_voreg_mni.nii.gz') % (i, i) for i in ids]
confounds_fns = [data_dir + '\\adhd\\data\\%s\\%s_regressors.csv' % (i, i) for i in ids]

In [ ]:
atlas_fn = 'C:\\Users\\sr1209\\Downloads\\nilearn_data\\MSDL_rois\\msdl_rois.nii'
label_fn = 'C:\\Users\\sr1209\\Downloads\\nilearn_data\\MSDL_rois\\msdl_rois_labels.csv'

In [ ]:
# Load atlas
masker = NiftiMapsMasker(maps_img=atlas_fn, 
                         standardize=True, 
                         detrend=True,
                         t_r=2.5,
                         low_pass=.1, 
                         high_pass=.01)

In [ ]:
# Get labels and coordinates
import csv
from collections import defaultdict

columns = defaultdict(list) # each value in each column is appended to a list
coords = []

index = 0
with open(label_fn) as f:
    reader = csv.DictReader(f) # read rows into a dictionary format
    for row in reader: # read a row as {column1: value1, column2: value2,...}
        for (k,v) in row.items(): # go over each column name and value 
            columns[k].append(v) # append the value into the appropriate list
                                 # based on column name k
        
        # Also store coordinates into list
        coords.append([float(columns['x'][index]),float(columns['y'][index]),float(columns['z'][index])])
        index = index + 1
                  
labels = columns['name']

In [ ]:
# Compute functional connectivity for each subject across all ROIs
pooled_subjects = []
adhd_subjects = []
control_subjects = []

index = 0
for img in func_fns:
    # Load data into numpy array (not necessary)
    # func_imgs[index] = nib.load(file).get_data()
    
    # Extract timeseries for each ROI
    time_series = masker.fit_transform(func_fns[index], confounds=confounds_fns[index])
    pooled_subjects.append(time_series)
    if group[index] == 1: # if subject is in adhd group
       adhd_subjects.append(time_series)
    elif group[index] == 0: # if subject is in control group
       control_subjects.append(time_series)

    # Update index
    index = index + 1

In [ ]:
# Specify correlation measures
pooled_corr_measure = ConnectivityMeasure(kind='correlation') 
adhd_corr_measure = ConnectivityMeasure(kind='correlation')
control_corr_measure = ConnectivityMeasure(kind='correlation')
# For partial correlation: ConnectivityMeasure(kind='partial correlation')

pooled_corr_matrices = pooled_corr_measure.fit_transform(pooled_subjects)
adhd_corr_matrices = adhd_corr_measure.fit_transform(adhd_subjects)
control_corr_matrices = control_corr_measure.fit_transform(control_subjects)

In [ ]:
def plot_matrices(matrices, matrix_kind, labels):
    n_matrices = len(matrices)
    fig = plt.figure(figsize=(n_matrices * 6, 6))
    for n_subject, matrix in enumerate(matrices):
        plt.subplot(1, n_matrices, n_subject + 1)
        matrix = matrix.copy()  # avoid side effects
        # Set diagonal to zero, for better visualization
        np.fill_diagonal(matrix, 0)
        vmax = np.max(np.abs(matrix))
        plotting.plot_matrix(matrix, vmin=-vmax, vmax=vmax, labels=labels, 
                             cmap='RdBu_r',figure=fig, colorbar=False)

In [ ]:
%matplotlib inline 
plot_matrices(adhd_corr_matrices, 'correlation', labels) # adhd subjects
mean_adhd_corr_matrix = adhd_corr_measure.mean_
plotting.plot_connectome(mean_adhd_corr_matrix,
                         coords,
                         edge_threshold="80%",
                         colorbar=True,
                         title='mean correlation over %s ADHD subjects' % len(adhd_corr_matrices))
plotting.show()

In [ ]:
plot_matrices(control_corr_matrices, 'correlation', labels) # controls
mean_control_corr_matrix = control_corr_measure.mean_
plotting.plot_connectome(mean_control_corr_matrix,
                         coords,
                         edge_threshold="80%",
                         colorbar=True,
                         title='mean correlation over %s control subjects' % len(control_corr_matrices))
plotting.show()

In [ ]:
mean_pooled_corr_matrix = pooled_corr_measure.mean_
plotting.plot_connectome(mean_pooled_corr_matrix,
                         coords,
                         edge_threshold="80%",
                         colorbar=True,
                         title='mean correlation over %s subjects' % len(pooled_corr_matrices))
plotting.show()

We can also view an interactive graph plot:

In [ ]:
# view = plotting.view_connectome(mean_pooled_corr_matrix, coords, threshold='80%')
# view.open_in_browser()

### Now, let's test a specific hypotheses:
- Decreased ACC (22) <--> PCC (33) functional connectivity in ADHD group than controls (Castellanos et al, 2008)

In [ ]:
# for c, value in enumerate(labels):
#     print(c, value)

In [ ]:
ACC_PCC = pd.DataFrame({"controls" : control_corr_matrices[0:3,22,33],
                        "adhd" : adhd_corr_matrices[0:3,22,33]})

In [ ]:
from scipy.stats import ttest_ind as ttest

(T_stat, p_val) = ttest(control_corr_matrices[0:3,22,33], adhd_corr_matrices[0:3,22,33])
print("T = %f, p = %f" % (T_stat, p_val))

#### Now, let's get fancy with some seaborn

In [ ]:
import seaborn as sns

In [ ]:
# Plot dACC <--> dPCC group analysis results
sns.set(style="whitegrid")
ax = sns.barplot(data=ACC_PCC )

In [ ]:
# Let's plot the correlation matrices using seaborn
# Throw matrices into dataframe
adhd_df = pd.DataFrame(mean_adhd_corr_matrix, columns = labels, index = labels)
control_df = pd.DataFrame(mean_control_corr_matrix, columns = labels, index = labels)

sns.set(style="white")

# Generate a mask for the upper triangle
mask = np.zeros_like(adhd_df, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Set up the matplotlib figure
fig = plt.figure(figsize = (50,50)) # width x height
ax1 = fig.add_subplot(3, 3, 1) # row, column, position
ax2 = fig.add_subplot(3, 3, 2)

sns.heatmap(data=adhd_df, ax=ax1, mask=mask, cmap=cmap, square=True, vmax=.3, cbar_kws={"shrink": .5})
sns.heatmap(data=control_df, ax=ax2, mask=mask, cmap="YlGnBu", square=True, vmax=.3, cbar_kws={"shrink": .5})